# Dashboard - Visual Board

## Name: Rohit Dev
## Matrikel Nr.- 411817

In [1]:
# Importing Libraries
import pandas as pd
import numpy as np

import dash
dash.__version__
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output,State

import plotly.graph_objects as go

import os

C:\Users\DigiBau\AppData\Local\Temp\ipykernel_25252\251499227.py:7: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\DigiBau\AppData\Local\Temp\ipykernel_25252\251499227.py:8: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
# Import the processes data (Data processing is done in Dashboad-Data Processing Jupyter Notebook)
processed_data=pd.read_csv('../data/processed/Dashboard_final_set.csv',sep=';')

In [3]:
processed_data

,index,date,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
0,0,2020-02-24,Afghanistan,5.0,NaN,5.0,NaN
1,1,2020-02-25,Afghanistan,5.0,NaN,5.0,NaN
2,2,2020-02-26,Afghanistan,5.0,inf,5.0,NaN
3,3,2020-02-27,Afghanistan,5.0,inf,5.0,NaN
4,4,2020-02-28,Afghanistan,5.0,inf,5.0,NaN
...,...,...,...,...,...,...,...
204535,204535,2022-07-23,Zimbabwe,256246.0,1.767080e+04,256240.8,17081.684444
204536,204536,2022-07-24,Zimbabwe,256254.0,1.385076e+04,256254.2,17082.648889
204537,204537,2022-07-25,Zimbabwe,256270.0,2.135472e+04,256273.8,15530.682828
204538,204538,2022-07-26,Zimbabwe,256284.0,1.708462e+04,256290.6,14080.926740


In [4]:
fig = go.Figure()

In [5]:
app = dash.Dash()
app.layout = html.Div([

    dcc.Markdown('''
    #  Enterprise Data Science - (Dashboard for COVID-19 Data)
    
    '''),

    dcc.Markdown('''
    ## Multi-Select Country for visualization
    '''),

    dcc.Dropdown(
        id='country_drop_down',
        options=[ {'label': each,'value':each} for each in processed_data['country'].unique()],
        value=['Sweden', 'Denmark','France'], # which are pre-selected
        multi=True
    ),

    dcc.Markdown('''
        ## Select Timeline of confirmed COVID-19 cases or the approximated doubling time
        '''),


    dcc.Dropdown(
    id='doubling_time',
    options=[
        {'label': 'Timeline Confirmed ', 'value': 'confirmed'},
        {'label': 'Timeline Confirmed Filtered', 'value': 'confirmed_filtered'},
        {'label': 'Timeline Doubling Rate', 'value': 'confirmed_DR'},
        {'label': 'Timeline Doubling Rate Filtered', 'value': 'confirmed_filtered_DR'},
    ],
    value='confirmed',
    multi=False
    ),

    dcc.Graph(figure=fig, id='main_window_slope')
])

In [6]:
# callback function
@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value'),
    Input('doubling_time', 'value')])
def update_figure(country_list,show_doubling):


    if 'doubling_rate' in show_doubling:
        my_yaxis={'type':"log",
               'title':'Approximated doubling rate over 3 days'
              }
    else:
        my_yaxis={'type':"log",
                  'title':'Confirmed Covid Cases (source: Our World in Data (OWID) - log-scale)'
              }


    traces = []
    for each in country_list:

        df_plot=processed_data[processed_data['country']==each]

        if show_doubling=='doubling_rate_filtered':
            df_plot=df_plot[['country','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date']].groupby(['country','date']).agg(np.mean).reset_index()
        else:
            df_plot=df_plot[['country','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date']].groupby(['country','date']).agg(np.sum).reset_index()
       #print(show_doubling)


        traces.append(dict(x=df_plot.date,
                                y=df_plot[show_doubling],
                                mode='markers+lines',
                                opacity=0.9,
                                name=each
                        )
                )

    return {
            'data': traces,
            'layout': dict (
                width=1280,
                height=720,

                xaxis={'title':'Timeline',
                        'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                      },

                yaxis=my_yaxis
        )
    }

In [ ]:
# Running the server
app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


# End